# Processamento do Arquivo de Vacinação 

Este documento é utilizado para processar o arquivo **XLSX** disponibilizado pela **Secretaria de Estado da Saúde do estado de Sergipe** com os quantitativos da vacinação do estado. Foi utilizada a biblioteca **[pandas](https://pandas.pydata.org/)** para carregar e processar o arquivo com a finalidade de gerar um arquivo .csv mais adequado para o processo de visualização de dados. Um exemplo de visualização destes dados está no Painel COVID-19 disponível [neste link](https://covid19.data2learning.com).

Diferentes tipos de arquivos já foram compartilhados pela secretaria. Esse processamento se refere ao arquivo disponibilizado em **04/03/21** (a secretaria voltou a usar o Excel após alguns dias disponibilizando somente pdf, o código para o PDF está no final deste notebook). O arquivo excel atual tem a seguinte estrutura de colunas: 

Estrutura do arquivo disponibilizado a partir de 03/03:

* Municípios
* Trabalhadores de Saúde
    * População
        * TOTAL
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
* Pessoas com 60 anos ou mais e deficientes Institucionalizados
    * População
        * TOTAL
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
* Índio Aldeado
    * População
        * TOTAL
    * 1ª Dose
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Aplicadas
        * COB
* Pessoas com 90 ou mais
    * Doses Enviadas
    * Doses Aplicadas
        * 1ª
        * COB
        * 2ª
        * COB
* Idosos de 80 a 89 anos
    * Doses enviadas
    * Doses Aplicadas
        * 80-84 anos
        * 85-89 anos
    * COB
* TOTAL DE DOSES ENVIADAS
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB

Totalizando 31 colunas com informações de população, doses enviadas, doses aplicadas e cobertura. 

Só para registro de documentação, deixo a seguir a estrutura do arquivo disponibilizado até 23/02. O código para processmennto é feito com base na versão do dia 02/03.

* Municípios
* Trabalhadores de Saúde
    * População
        * TOTAL
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
* Pessoas com 60 anos ou mais e deficientes Institucionalizados
    * População
        * TOTAL
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
* Índio Aldeado
    * População
        * TOTAL
    * 1ª Dose
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Aplicadas
        * COB
* Pessoas com 90 ou mais
    * Doses Enviadas
    * Doses Aplicadas
        * 1ª
        * COB
        * 2ª
        * COB
* TOTAL DE DOSES ENVIADAS
    * 1ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
    * 2ª Dose
        * Doses Enviadas
        * Doses Aplicadas
        * COB
        


## Etapas do Processamento - EXCEL

Para trabalhar melhor com as informações, foi preciso fazer um organização das informações disponbilizadas. Apesar do pandas lê facilmente arquivos do Excel, a estrutura em níveis das colunas no documento dificulta um pouco o trabalho no DataFrame gerado. Além disso, o documento possui algumas linhas com informações de título do documento como descrição do Governo, secretaria e coordenação responsável pelo documento. Como a idéia é gerar um arquivo .CSV que seja de fácil leitura, essas informações precisam ser removidas do documento.

As etapas do processamento foram: 

1. Definir uma nomeclatura para cada coluna para excluir a hierarquia criada no arquivo excel. 
2. Definir as colunas de interesse para geração do CSV. 
3. Carregar o XLSX e gerar o arquivo final.

Segue a execução das etapas. 

In [1]:
# Import da biblioteca pandas

import pandas as pd

### Nomeclatura das colunas

As colunas foram separadas em 4 grupos que corresponde a cada grupo apto para vacinação, conforme o arquivo do excel disponibilizado. São eles:

* Grupo 1: Trabalhadores da Saúde
* Grupo 2: Pessoas com 60 anos ou mais e deficientes Institucionalizados
* Grupo 3: Índio Aldeado
* Grupo 4: Pessoas com 90 ou mais
* Grupo 5: Idosos de 80 a 89 anos que subdivide em 80-84 anos e 85-89 anos

Com isso foram definidos  os nomes de cada coluna de acordo com o que estava disponível no arquivo original para cada grupo. Por exemplo, para o grupo 1 é informado a população, o quantitativo de doses envidas, o quantitativo de doses aplicadas e a cobertura. Essas últimas 3 informações para cada dose. Alguns grupos não possui todas essas informações. Por conta disso, a definição foi feita manualmente, como consta no código a seguir: 

In [2]:
# Lista com os nomes das colunas do documento excel.

list_columns = [
    'Municipio',
    'Grupo1_Populacao',
    'Grupo1_Dose1_Enviadas',
    'Grupo1_Dose1_Aplicadas',
    'Grupo1_Dose1_Cobertura',
    'Grupo1_Dose2_Enviadas',
    'Grupo1_Dose2_Aplicadas',
    'Grupo1_Dose2_Cobertura',
    'Grupo2_Populacao',
    'Grupo2_Dose1_Enviadas',
    'Grupo2_Dose1_Aplicadas',
    'Grupo2_Dose1_Cobertura',
    'Grupo2_Dose2_Enviadas',
    'Grupo2_Dose2_Aplicadas',
    'Grupo2_Dose2_Cobertura',
    'Grupo3_Populacao',
    'Grupo3_Dose1_Aplicadas',
    'Grupo3_Dose1_Cobertura',
    'Grupo3_Dose2_Aplicadas',
    'Grupo3_Dose2_Cobertura',
    'Grupo4_DosesEnviadas',
    'Grupo4_Doses1_Aplicadas',
    'Grupo4_Doses1_Cobertura',
    'Grupo4_Doses2_Aplicadas',
    'Grupo4_Doses2_Cobertura',
    'Grupo5_DosesEnviadas',
    'Grupo5_Doses_Aplicadas_8084',
    'Grupo5_Doses_Aplicadas_8589',
    'Grupo5_Cobertura',
    'Total_Dose1_Enviadas',
    'Total_Dose1_Aplicadas',
    'Total_Dose1_Cobertura',
    'Total_Dose2_Enviadas',
    'Total_Dose2_Aplicadas',
    'Total_Dose2_Cobertura'
]

### Colunas de interesse 

Nem todas as colunas foram incluídas no CSV final. A coluna cobertura possui informação em %. Em alguns testes que foram feitos, o pandas não gerava essa coluna de forma adequada quando era carregado. Como essa é uma informação fácil de calcular pela relação entre as doses envidas e aplicadas, foi preferido retira-la do documento final. No processo de visualização essa informação pode ser facilmente calculada e exibida. Isso é feito no Painel linkado no início do documento. 

In [3]:
# Colunas definidas para o documento final

final_columns = [
    'Municipio',
    'Grupo1_Populacao',
    'Grupo1_Dose1_Enviadas',
    'Grupo1_Dose1_Aplicadas',
    'Grupo1_Dose2_Enviadas',
    'Grupo1_Dose2_Aplicadas',
    'Grupo2_Populacao',
    'Grupo2_Dose1_Enviadas',
    'Grupo2_Dose1_Aplicadas',
    'Grupo2_Dose2_Enviadas',
    'Grupo2_Dose2_Aplicadas',
    'Grupo3_Populacao',
    'Grupo3_Dose1_Aplicadas',
    'Grupo3_Dose2_Aplicadas',
    'Grupo4_DosesEnviadas',
    'Grupo4_Doses1_Aplicadas',
    'Grupo4_Doses2_Aplicadas',
    'Grupo5_DosesEnviadas',
    'Grupo5_Doses_Aplicadas_8084',
    'Grupo5_Doses_Aplicadas_8589',
    'Total_Dose1_Enviadas',
    'Total_Dose1_Aplicadas',
    'Total_Dose2_Enviadas',
    'Total_Dose2_Aplicadas',
]

### Carregar o XLSX e gerar o arquivo final

Os trechos de códigos a seguir servem para carregar o arquivo .xslx e gerar o arquivo CSV final. Como já definimos os nomes das colunas manualmente, podemos descartar as 8 primeiras linhas do documento que possui informações de título e nome das colunas de forma hierárquica. A explicação de cada etapa está nos comentários.

In [5]:
'''
Carrega o arquivo .xlsx descartando as 8 primeiras linhas (skiprows=8) que correspondem
as linhas de informações dos governos e títulos das colunas. O parâmetro names força que 
o dataframe gerado tenha os nomes das colunas definidas manualmente. 
''' 
data_final = pd.read_excel("data/Doses-aplicadas-04.03.xlsx", skiprows=8,names=list_columns)

'''
.head(-1) exclui a última linha do documento que possui o total de Sergipe. Essa informação
foi retirada porque é facilmente calcula a partir da soma de cada coluna. Isso pode ser feito
no momento de gerar as visualizações
''' 
data_final = data_final.head(-3)

'''
Gera o datframe com as colunas selecinadas para o documento final 
'''
data_final = data_final[final_columns]

'''
Alguns municípios possui valores em branco para algumas colunas. Quando o dataframe é carregado,
esses valores são reconhecidos como NULOS. O código a seguir substitui por 0.
'''
data_final = data_final.fillna(0)

'''
Força que as colunas que possui informações da vacinação seja do tipo inteiro.
final_columns[1:] pega da segunda coluna até a última, que são as colunas do tipo inteiro. 
'''
data_final[final_columns[1:]] = data_final[final_columns[1:]].astype(int)

'''
Visualiza o dataframe final 
''' 
data_final

,Municipio,Grupo1_Populacao,Grupo1_Dose1_Enviadas,Grupo1_Dose1_Aplicadas,Grupo1_Dose2_Enviadas,Grupo1_Dose2_Aplicadas,Grupo2_Populacao,Grupo2_Dose1_Enviadas,Grupo2_Dose1_Aplicadas,Grupo2_Dose2_Enviadas,...,Grupo4_DosesEnviadas,Grupo4_Doses1_Aplicadas,Grupo4_Doses2_Aplicadas,Grupo5_DosesEnviadas,Grupo5_Doses_Aplicadas_8084,Grupo5_Doses_Aplicadas_8589,Total_Dose1_Enviadas,Total_Dose1_Aplicadas,Total_Dose2_Enviadas,Total_Dose2_Aplicadas
0,Amparo de São Francisco,64,52,52,42,42,0,0,0,0,...,10,8,0,40,2,10,102,72,52,42
1,Aquidabã,397,297,259,257,231,17,17,18,17,...,98,83,0,320,0,31,732,391,372,247
2,Aracaju,32381,23639,16075,10939,10789,230,230,243,230,...,2061,2300,0,6760,0,5054,32690,23672,13230,11005
3,Arauá,219,162,142,142,85,0,0,0,0,...,30,33,20,110,63,44,302,282,172,105
4,Areia Branca,387,281,230,251,210,0,0,0,0,...,57,47,16,180,12,109,518,398,308,226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Siriri,157,112,108,102,100,0,0,0,0,...,31,31,28,100,72,40,243,251,133,128
71,Telha,107,79,69,69,60,0,0,0,0,...,10,10,0,30,15,7,119,101,79,60
72,Tobias Barreto,1091,803,839,373,395,33,33,33,33,...,212,212,212,700,0,380,1748,1464,618,640
73,Tomar do Geru,161,114,111,104,94,0,0,0,0,...,42,26,11,140,8,40,296,185,146,105


O código a seguir gera o arquivo final no formato CSV. O atributo `index=False` força que o documento não tenha um índice por linha. Essa informação, neste caso, não é necessária para os processos de visualização.

In [6]:
data_final.to_csv("output/relatorio_vacina_sergipe.csv", index=False)

## Etapa do Processamento - PDF

A partir de 24 de Fevereiro, a secretaria passou a divulgar os dados em um arquivo PDF. O processamento desse tipo de arquivo é bem mais complicado. O código a seguir utiliza a biblioteca [tabula-py](https://tabula-py.readthedocs.io/en/latest/index.html) para converter o PDF em um DataFrame pandas. No entanto, o reconhecimento dos tipos não é 100%. Foi preciso algumas alterações pontuais para converter os números nos tipos corretos. Essas alterações podem mudar a medida que o arquivo mude. 

A estrutura de colunas se manteve a mesma do início do documento. 

O código a seguir foi aplicado ao documento disponibilizado em **01/03/2021**.

**Em 02/03/21, a secretaria voltou a divulgar os dados no formato excel**


In [7]:
# Instala a biblioteca tabula-py
!pip install tabula-py

     |████████████████████████████████| 11.7 MB 1.9 MB/s eta 0:00:01    |██████████████▏                 | 5.2 MB 1.6 MB/s eta 0:00:05     |██████████████▊                 | 5.4 MB 1.6 MB/s eta 0:00:05     |███████████████▎                | 5.6 MB 1.6 MB/s eta 0:00:04


In [1]:
# Imports necessários 

import tabula
import pandas as pd

'''
Lê o arquivo pdf passando alguns parâmetros do pandas para tratar a leitura dos números de forma correta. 
Ainda assim, alguns números não foram lidos nos tipos corretos. As correções são feitas nas linhas a seguir
''' 
df_tabula = tabula.read_pdf("data/Doses-aplicadas-01.03.pdf", pages='all',lattice=True,pandas_options={'thousands': '.', 'decimal': ','}, multiple_tables=False)

In [ ]:
# Lista com os nomes das colunas do documento excel.

list_columns = [
    'Municipio',
    'Grupo1_Populacao',
    'Grupo1_Dose1_Enviadas',
    'Grupo1_Dose1_Aplicadas',
    'Grupo1_Dose1_Cobertura',
    'Grupo1_Dose2_Enviadas',
    'Grupo1_Dose2_Aplicadas',
    'Grupo1_Dose2_Cobertura',
    'Grupo2_Populacao',
    'Grupo2_Dose1_Enviadas',
    'Grupo2_Dose1_Aplicadas',
    'Grupo2_Dose1_Cobertura',
    'Grupo2_Dose2_Enviadas',
    'Grupo2_Dose2_Aplicadas',
    'Grupo2_Dose2_Cobertura',
    'Grupo3_Populacao',
    'Grupo3_Dose1_Aplicadas',
    'Grupo3_Dose1_Cobertura',
    'Grupo3_Dose2_Aplicadas',
    'Grupo3_Dose2_Cobertura',
    'Grupo4_DosesEnviadas',
    'Grupo4_Doses1_Aplicadas',
    'Grupo4_Doses1_Cobertura',
    'Grupo4_Doses2_Aplicadas',
    'Grupo4_Doses2_Cobertura',
    'Total_Dose1_Enviadas',
    'Total_Dose1_Aplicadas',
    'Total_Dose1_Cobertura',
    'Total_Dose2_Enviadas',
    'Total_Dose2_Aplicadas',
    'Total_Dose2_Cobertura'
]

In [ ]:
# Colunas definidas para o documento final

final_columns = [
    'Municipio',
    'Grupo1_Populacao',
    'Grupo1_Dose1_Enviadas',
    'Grupo1_Dose1_Aplicadas',
    'Grupo1_Dose2_Enviadas',
    'Grupo1_Dose2_Aplicadas',
    'Grupo2_Populacao',
    'Grupo2_Dose1_Enviadas',
    'Grupo2_Dose1_Aplicadas',
    'Grupo2_Dose2_Enviadas',
    'Grupo2_Dose2_Aplicadas',
    'Grupo3_Populacao',
    'Grupo3_Dose1_Aplicadas',
    'Grupo3_Dose2_Aplicadas',
    'Grupo4_DosesEnviadas',
    'Grupo4_Doses1_Aplicadas',
    'Grupo4_Doses2_Aplicadas',
    'Total_Dose1_Enviadas',
    'Total_Dose1_Aplicadas',
    'Total_Dose2_Enviadas',
    'Total_Dose2_Aplicadas',
]

In [5]:
# Lê o dataframe resultante do tabula eliminando as 3 primeiras linhas
df_ = df_tabula[0].iloc[3:]

# Define os nomes das colunas
df_.columns = list_columns

# Reseta os index
df_.reset_index(inplace=True)

#Seleciona somente as colunas de interesse (eliminei as colunas de cobertura que podem ser calculadas)
df_ = df_[final_columns]

# Coloca zero nos valores NAN
df_ = df_.fillna(0)

# Corrige problema específico com uma célula de Porto da Folha que reconheceu 260 como 2 60.
df_['Grupo3_Populacao'] = df_['Grupo3_Populacao'].str.replace(' ', '')

# isso pode gerar novos valores NAN. Substituo por zero novamente.
df_ = df_.fillna(0)

# Elimina a última linha total de Sergipe (campo que pode ser calculado também)
df_ = df_.head(-1)

df_

,Municipio,Grupo1_Populacao,Grupo1_Dose1_Enviadas,Grupo1_Dose1_Aplicadas,Grupo1_Dose2_Enviadas,Grupo1_Dose2_Aplicadas,Grupo2_Populacao,Grupo2_Dose1_Enviadas,Grupo2_Dose1_Aplicadas,Grupo2_Dose2_Enviadas,...,Grupo3_Populacao,Grupo3_Dose1_Aplicadas,Grupo3_Dose2_Aplicadas,Grupo4_DosesEnviadas,Grupo4_Doses1_Aplicadas,Grupo4_Doses2_Aplicadas,Total_Dose1_Enviadas,Total_Dose1_Aplicadas,Total_Dose2_Enviadas,Total_Dose2_Aplicadas
0,Amparo de São Francisco,64,42,46,42,42,0,0,0,0,...,0,0,0,10,6,0,52.0,52.0,42.0,42.0
1,Aquidabã,397,257,258,257,223,17,17,18,17,...,0,0,0,98,83,0,372.0,359.0,274.0,239.0
2,Aracaju,32.381,23.639,15.655,10.939,10.538,230,230,240,230,...,0,0,0,2.061,2291,0,25930.0,18186.0,11169.0,10736.0
3,Arauá,219,142,142,142,63,0,0,0,0,...,0,0,0,30,30,0,172.0,172.0,142.0,63.0
4,Areia Branca,387,251,227,251,193,0,0,0,0,...,0,0,0,57,40,0,308.0,267.0,251.0,193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Siriri,157,102,103,102,97,0,0,0,0,...,0,0,0,31,31,0,133.0,134.0,102.0,97.0
71,Telha,107,69,69,69,59,0,0,0,0,...,0,0,0,10,10,0,79.0,79.0,69.0,59.0
72,Tobias Barreto,1.091,713,721,373,394,33,33,33,33,...,0,0,0,212,212,0,958.0,966.0,406.0,427.0
73,Tomar do Geru,161,104,111,104,91,0,0,0,0,...,0,0,0,42,24,0,146.0,135.0,104.0,91.0


In [6]:
'''
Observe que algumas colunas estão reconhecendo números como 32.381. Na verdade esse número é uma string. Essas colunas específicas foram 
tratadas de forma específica substituindo o . por vazio para que possa ser tranformado em número inteiro. Caso não fizesse isso, 
o número 32.381 seria convertido para 32 (caso convertesse em inteiro) ou 32,381 (caso float). 
'''

specific_problems = [
    'Grupo1_Populacao',
    'Grupo1_Dose1_Enviadas',
    'Grupo1_Dose1_Aplicadas',
    'Grupo1_Dose2_Enviadas',
    'Grupo1_Dose2_Aplicadas',
    'Grupo2_Populacao',
    'Grupo2_Dose1_Enviadas',
    'Grupo2_Dose1_Aplicadas',
    'Grupo2_Dose2_Enviadas',
    'Grupo2_Dose2_Aplicadas',
    'Grupo3_Populacao',
    'Grupo3_Dose1_Aplicadas',
    'Grupo3_Dose2_Aplicadas',
    'Grupo4_DosesEnviadas',
]

for column in specific_problems:
    df_[column] = df_[column].str.replace('.', '')

In [7]:
# A ação anterior pode gerar colunas NAN. Substituo novamente por 0.
df_ = df_.fillna(0)

# Converte as colunas para inteiro.
df_[final_columns[1:]] = df_[final_columns[1:]].astype(int)

# Imprime o data frame final
df_

,Municipio,Grupo1_Populacao,Grupo1_Dose1_Enviadas,Grupo1_Dose1_Aplicadas,Grupo1_Dose2_Enviadas,Grupo1_Dose2_Aplicadas,Grupo2_Populacao,Grupo2_Dose1_Enviadas,Grupo2_Dose1_Aplicadas,Grupo2_Dose2_Enviadas,...,Grupo3_Populacao,Grupo3_Dose1_Aplicadas,Grupo3_Dose2_Aplicadas,Grupo4_DosesEnviadas,Grupo4_Doses1_Aplicadas,Grupo4_Doses2_Aplicadas,Total_Dose1_Enviadas,Total_Dose1_Aplicadas,Total_Dose2_Enviadas,Total_Dose2_Aplicadas
0,Amparo de São Francisco,64,42,46,42,42,0,0,0,0,...,0,0,0,10,6,0,52,52,42,42
1,Aquidabã,397,257,258,257,223,17,17,18,17,...,0,0,0,98,83,0,372,359,274,239
2,Aracaju,32381,23639,15655,10939,10538,230,230,240,230,...,0,0,0,2061,2291,0,25930,18186,11169,10736
3,Arauá,219,142,142,142,63,0,0,0,0,...,0,0,0,30,30,0,172,172,142,63
4,Areia Branca,387,251,227,251,193,0,0,0,0,...,0,0,0,57,40,0,308,267,251,193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Siriri,157,102,103,102,97,0,0,0,0,...,0,0,0,31,31,0,133,134,102,97
71,Telha,107,69,69,69,59,0,0,0,0,...,0,0,0,10,10,0,79,79,69,59
72,Tobias Barreto,1091,713,721,373,394,33,33,33,33,...,0,0,0,212,212,0,958,966,406,427
73,Tomar do Geru,161,104,111,104,91,0,0,0,0,...,0,0,0,42,24,0,146,135,104,91


In [8]:
# Salva o dataframe final
df_.to_csv("output/relatorio_vacina_sergipe.csv", index=False)

Desenvolvido por **@adolfoguimaraes**.

Qualquer dúvida sobre o código, só entrar em contato: 
* Twitter: https://twitter.com/adolfoguimaraes
* Instagram: https://instagram.com/profadolfoguimaraes
* GitHub: https://github.com/adolfoguimaraes

*Atenção: Esse trabalho é feito de forma totalmente independente e não tem nenhuma ligação com a secretaria que disponibiliza os dados.*